In [ ]:
import urllib.request
import io
import openpyxl
import sqltables.sqlite3
import plotly.graph_objs as go

In [ ]:
db = sqltables.sqlite3.Database()

In [ ]:
def fetch_date(date):
    url = f"https://www.regelleistung.net/apps/cpp-publisher/api/v1/download/tenders/anonymousresults?date={date}&exportFormat=xlsx&market=CAPACITY&productTypes=FCR"
    with urllib.request.urlopen(url) as fh:
        byte_data = fh.read()
    buf = io.BytesIO(byte_data)
    wb = openpyxl.open(buf)
    [header, *data] = wb.active.values
    return header, data

In [ ]:
all_dates = [f"{y}-{m:02d}-01" for y in ["2022", "2023"] for m in range(1, 13)]

In [ ]:
import calendar
cal = calendar.Calendar()
all_dates = [w[0].isoformat() for y in [2020, 2021, 2022, 2023] for m in range(1, 13) for w in cal.monthdatescalendar(y, m)]
all_dates

In [ ]:
dates = [x for x in all_dates if x <= "2023-06-15" and x > "2020-06-15"]
dates

In [ ]:
from tqdm.notebook import tqdm
date_tables = {}
for date in tqdm(dates):
    if date in date_tables:
        continue
    header, rows = fetch_date(date)
    date_tables[date] = db.create_table(column_names=header, rows=rows)

In [ ]:
def merge_tables(table1, table2=None):
    if table2 is None:
        return table1
    col1 = iter(table1).column_names
    col2 = iter(table2).column_names
    columns = [c for c in col1 if c in col2]
    column_str = ",".join(db.quote_name(c) for c in columns)
    return table1.table(f"""select {column_str} from _ union select {column_str} from _2""", bindings={"_2": table2})

In [ ]:
from functools import reduce

In [ ]:
offers = reduce(merge_tables, tqdm(date_tables.values()))

In [ ]:
# offers = merge_tables(date_tables[dates[0]], date_tables[dates[1]])

In [ ]:
all_keys = offers.view("""select distinct date_from, type_of_reserves, product from _""")
all_keys

In [ ]:
keys = all_keys.view("""select * from _ where product = 'NEGPOS_08_12'""")

In [ ]:
marginal_curves = {}
for k in keys:
    product_offers = offers.table(""" select * from _ where product = ? and date_from = ?""", parameters=[k.PRODUCT, k.DATE_FROM])
    marginal_curve = product_offers.view("""
select
    sum("OFFERED_CAPACITY_[MW]") over 
        (order by "OFFERED_CAPACITY_PRICE_[EUR/MW]" rows between unbounded preceding and current row) as marginal_capacity,
    "OFFERED_CAPACITY_PRICE_[EUR/MW]"
from _
""")
    marginal_curves[k] = marginal_curve

In [ ]:
import math
import datetime
traces = []
N = len(marginal_curves)
for i, (k, marginal_curve) in enumerate(marginal_curves.items()):
    [x, y] = zip(*marginal_curve)
    grey = int(255*(1-i/N))
    traces.append({"x": x, "y": y, "name": ",".join(k),
#                    "line": {"shape": "hv"}
#                    "line": {"color": f"rgb({grey}, {grey}, {grey})"}

In [ ]:
import math
import datetime
traces = []
N = len(marginal_curves)
for i, (k, marginal_curve) in enumerate(marginal_curves.items()):
    [x, y] = zip(*marginal_curve)
    grey = int(255*(1-i/N))
    traces.append({"x": x, "y": y, "name": ",".join(k),
#                    "line": {"shape": "hv"}
#                    "line": {"color": f"rgb({grey}, {grey}, {grey})"}
                  })

In [ ]:
layout = {
    "updatemenus": [dict(
            type="buttons",
            x=-0.15,
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None]),
                     dict(label="Pause",
                          method="animate",
                          args=[None,
                               {"frame": {"duration": 0, "redraw": True},
                                "mode": "immediate",
                                "transition": {"duration": 0}}],
                         )])],
    "plot_bgcolor": "white",
    "showlegend": False,
    "yaxis": {"range": [0, 300], "title": "EUR", "gridcolor": "#f2f2f2", "gridwidth": 1, "linecolor": "#f2f2f2", "linewidth": 1},
    "xaxis": {"range": [0, 1500], "title": "MW", "gridcolor": "#f2f2f2", "gridwidth": 1, "linecolor": "#f2f2f2", "linewidth": 1},
#     "title": {"x": 0.4, "y": 0.4, "text": ""}
    "title": {"x": 0.5, "y": 0.9, "text": ""}
}
empty_traces = [{"x": [], "y": [], "name": t["name"]} for t in traces]
frames = []
for i in range(1, len(traces)+1):
    frame_traces = [{**t} for t in traces[:i]]
    for k, trace in enumerate(frame_traces):
        j = i - k - 1
        val = (1-math.exp(-15*j/N))
        grey = int(255*(1-math.exp(-8*j/N)))
#         trace.setdefault("line", {})["color"] = f"rgb({grey}, {grey}, {grey})"
        trace.setdefault("line", {})["color"] = f"#2f3eea"
        trace["opacity"] = max(0.05, 1-val)
    frame_dt = datetime.datetime.fromisoformat(frame_traces[-1]["name"][:10])
    frame_title = frame_dt.strftime("%A,\n%d %B %Y")
    frames.append({"data": frame_traces, "layout": {"title": {"text": frame_title}}})
layout["title"]["text"] = frames[0]["layout"]["title"]["text"]
go.Figure(data=empty_traces, layout=layout, frames=frames)